In [1]:
import os   

In [2]:
%pwd

'c:\\Users\\ParagJadhav\\Downloads\\ML_Projects\\red_wine\\red_wine_quality_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ParagJadhav\\Downloads\\ML_Projects\\red_wine\\red_wine_quality_prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    features_column: list
    target_column: str
    preprocessor_name: str

In [6]:
from src.mlproject.constant import *
from src.mlproject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        schema_feature = self.schema.COLUMNS
        schema_target =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            features_column = schema_feature,
            target_column = schema_target.name,
            preprocessor_name = config.preprocessor_name,
        )

        return data_transformation_config

In [8]:
import os
from src.mlproject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from mlproject.utils.common import save_bin


from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_spliting(self):
        logger.info("Started the Spliting the data into training and test sets")

        data = pd.read_csv(self.config.data_path)
        df_train, df_test = train_test_split(data, test_size = 0.2, random_state = 42)

        df_train.to_csv(os.path.join(self.config.root_dir, "df_train.csv"),index = False)
        df_test.to_csv(os.path.join(self.config.root_dir, "df_test.csv"),index = False)

        logger.info("Successfully splited data into training and test sets")
        logger.info(f'Shape of Training dataset df_train.shape: {df_train.shape}')
        logger.info(f'Shape of Testing dataset df_test.shape: {df_test.shape}')

        return df_train, df_test
        
    
    def get_data_transformer_object(self):

        logger.info("Starting Pipeline construction")
        features_column = self.config.features_column
        target_column = self.config.target_column

        categorical_columns = [col for col, dtype in features_column.items() if dtype == 'obj' and col !=  target_column]
        numerical_columns = [col for col, dtype in features_column.items() if dtype != 'obj' and col != target_column]


        num_pipeline=Pipeline(steps=[
            ("imputer",SimpleImputer(strategy='median')),
            ('scalar',StandardScaler()),
            ('TruncatedSVD', TruncatedSVD(5))
            ])
        
        cat_pipeline=Pipeline(steps=[
            ("imputer",SimpleImputer(strategy="most_frequent")),
            ("one_hot_encoder",OneHotEncoder(handle_unknown='ignore')),
            ("scaler",StandardScaler(with_mean=False)),
            ('TruncatedSVD', TruncatedSVD(5))
            ])

        logger.info(f"Categorical Columns: {categorical_columns}")
        logger.info(f"Numerical Columns: {numerical_columns}")

        preprocessor=ColumnTransformer(
            [
                ("num_pipeline",num_pipeline,numerical_columns),
                ("cat_pipeline",cat_pipeline,categorical_columns)
            ]
            )
        
        return preprocessor
    

    def initiate_data_transormation(self):

        logger.info("Started spliting data into X_train, y_train and X_test, y_test")
        df_train, df_test = self.train_test_spliting() 
        target_column = self.config.target_column
        

        X_train = df_train.drop(columns = [target_column], axis=1)
        y_train = df_train[target_column]

        X_test = df_test.drop(columns = [target_column], axis=1)
        y_test = df_test[target_column]

        logger.info("Successfully splited data into X_train, y_train and X_test, y_test")
        logger.info(f'Shape of X_train: {X_train.shape}')
        logger.info(f'Shape of y_train:  {y_train.shape}')
        logger.info(f'Shape of X_test:  {X_test.shape}')
        logger.info(f'Shape of y_test:  {y_test.shape}')

        
        logger.info("Reading the preprocessor pipeline object for training")
        preprocessing_obj=self.get_data_transformer_object()

        logger.info("Applying Preprocessing on training and test dataframe")
        input_feature_train_arr = preprocessing_obj.fit_transform(X_train)
        input_feature_test_arr = preprocessing_obj.transform(X_test)


        train_arr = np.c_[input_feature_train_arr, np.array(y_train)]
        test_arr = np.c_[input_feature_test_arr, np.array(y_test)]

        logger.info(f"Saved preprocessing object")        
        save_bin(preprocessing_obj, os.path.join(self.config.root_dir, self.config.preprocessor_name))
    


In [12]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config=data_transformation_config)
data_transformation.train_test_spliting()
data_transformation.get_data_transformer_object()
data_transformation.initiate_data_transormation()

[2024-05-17 09:22:10,517: line_number-45: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-17 09:22:10,520: line_number-45: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-17 09:22:10,523: line_number-45: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-17 09:22:10,523: line_number-83: INFO: common: created directory at: artifacts]
[2024-05-17 09:22:10,525: line_number-83: INFO: common: created directory at: artifacts/data_transformation]
[2024-05-17 09:22:10,525: line_number-6: INFO: 3254848639: Started the Spliting the data into training and test sets]
[2024-05-17 09:22:10,544: line_number-14: INFO: 3254848639: Successfully splited data into training and test sets]
[2024-05-17 09:22:10,544: line_number-15: INFO: 3254848639: Shape of Training dataset df_train.shape: (1279, 12)]
[2024-05-17 09:22:10,545: line_number-16: INFO: 3254848639: Shape of Testing dataset df_test.shape: (320, 12)]
[2024-05-17 09:22:10,546: line_number